# Train YOLO

In [ ]:
!pip install ultralytics

!wget -qO- https://github.com/direito-a-sombra/bus-view/releases/download/main-6df4357/imgs.tar.gz | tar xz -C ./data/audio/
!wget -q https://raw.githubusercontent.com/direito-a-sombra/bus-view/refs/heads/main/data/boxes.json

In [ ]:
import json

from os import listdir, makedirs, path
from shutil import copy2
from sklearn.model_selection import train_test_split

In [ ]:
# Get list of all files in training directories

IMG_DIR = "./imgs/training/bus_stop"

dirs = sorted([d for d in listdir(IMG_DIR) if path.isdir(f"{IMG_DIR}/{d}")])
dir2files = {d : sorted([f for f in listdir(f"{IMG_DIR}/{d}") if f.endswith("jpg")]) for d in dirs}

files2dir = {}

for d, fs in dir2files.items():
  for f in fs:
    files2dir[f] = d

In [ ]:
# Prep Yolo dataset

IMG_DIR = "./imgs/training/bus_stop"

DATASET_DIR = "./datasets/bus-view"
makedirs(f"{DATASET_DIR}/images/train", exist_ok=True)
makedirs(f"{DATASET_DIR}/images/val", exist_ok=True)
makedirs(f"{DATASET_DIR}/labels/train", exist_ok=True)
makedirs(f"{DATASET_DIR}/labels/val", exist_ok=True)

LABEL2ID = {
  "bus_stop": 0,
  "bus_stop_sign": 1,
}
ID2LABEL = { id:label for label,id in LABEL2ID.items() }

with open("./p5/ImageTagger/boxes.json", "r") as ifp:
  box_info = json.load(ifp)

tfs, vfs = train_test_split(list(box_info.keys()), test_size=0.2, random_state=1010)

tfs = set(tfs)
vfs = set(vfs)

for f, objs in box_info.items():
  split = "train" if f in tfs else "val"
  src_dir = files2dir[f]
  src_file = f"{IMG_DIR}/{src_dir}/{f}"
  dst_file = f"{DATASET_DIR}/images/{split}/{f}"
  copy2(src_file, dst_file)

  with open(f"{DATASET_DIR}/labels/{split}/{f.replace('.jpg', '.txt')}", "w") as ofp:
    for label, (x0,y0,x1,y1) in objs.items():
      cx, cy = int((x0 + x1) // 2) / 640, int((y0 + y1) // 2) / 640
      w, h = abs(int(x1 - x0)) / 640, abs(int(y1 - y0)) / 640
      ofp.write(f"{LABEL2ID[label]} {cx} {cy} {w} {h}\n")

with open(f"./datasets/bus-view.yaml", "w") as ofp:
  ofp.write(f"path: bus-view\n")
  ofp.write(f"train: images/train\n")
  ofp.write(f"val: images/val\n\n")
  ofp.write(f"names:\n")
  for id,label in ID2LABEL.items():
    ofp.write(f"  {id}: {label}\n")

In [ ]:
from ultralytics import YOLO

model = YOLO("yolo11x.pt")

In [ ]:
results = model.train(data="bus-view.yaml", epochs=64, imgsz=(640, 640), device="cuda")